# Efficiency: memory hierarchy; locality
_COSC 208, Introduction to Computer Systems, 2023-04-05_

## Announcements
* Project 4 due Thursday, April 13

## Outline
* Memory hierarchy
* Better use of registers
* Temporal vs. spatial locality

## No warm-up – Happy Wednesday!

## Memory hierarchy

* Compares various forms of storage in terms of
    * Access latency
    * Capacity
    * Cost
    * Volatility
* Access latency
    * Let's consider a 1hz CPU, which means 1 cycle = 1 second
    * Registers — 1 cycle = 1 second
    * Caches — ~10 cycles = ~10 seconds
    * Main memory — ~100 cycles = ~2 minutes
    * Solid-state drive — ~1 million cycles = ~11.5 days
    * Hard (i.e., traditional) disk drive — ~10 million cycles = ~115 days
    * Remote (i.e., network) storage — ~20ms = ~2 years
* Storage capacity
    * Let's assume 1 byte = 1mL
    * Registers — 30 * 8B = ~250mL = ~1 cup
    * Caches (Core i7 in MacBook Pro)
        * L1 — 32KB + 32KB = 64L = ~1 tank of gas
        * L2 — 512KB * 4 cores = 2048L = ~7 bathtubs
    * Main memory = 32GB (in MacBook Pro) = ~13 olympic swimming pools
    * SSD = 1TB (in MacBook Pro) = ~Lake Moraine
* Cost
    * 2 x 16GB DRAM = ~$100 = $3.12 per GB
    * 1TB SSD = $80 = $0.08 per GB
    * 2TB HDD = $60 = $0.03 per GB
* Volatility
    * Primary storage (registers, caches, and main memory) — volatile (i.e., data is lost if power is lost)
    * Secondary storage (SSD, HDD, network storage) — non-volatile (i.e., data is preserved if power is lost)

## Data movement

* Recall: _How does data move between the CPU, main memory, and secondary storage in the von Neumann Architecture?_ — bus
* _Why does data move between registers and main memory?_ — not enough room in registers to store all values used by a program at runtime
* _How can we move less data (i.e., perform fewer loads and stores)?_
    * Better use of registers
    * Leverage locality
        * Add additional memory to the CPU — i.e., a cache
        * Optimize code to improve locality

## Better use of registers

* Loads and stores are unnecessary when the value of a register is not changed between store and load instructions involving the same register and memory address
    * Example load and store which is _unnecessary_
        ```
        str x0, [sp,#4]
        ldr x0, [sp,#4]
        ```
    * Example load and store which is _necessary_
        ```
        str w0, [sp,#4]
        mov w0, #0x1
        str w0, [sp]
        ldr w0, [sp,#4]
        ```

* Q1: _Cross-out unnecessary loads and stores for each of the following snippets of assembly code_

```
000000000000088c <interest_due>:
    88c:    sub    sp, sp, #0x20
    890:    str    w0, [sp, #12]    XXXXX
    894:    str    w1, [sp, #8]     XXXXX
    898:    ldr    w0, [sp, #12]    XXXXX
    89c:    ldr    w1, [sp, #8]     XXXXX
    8a0:    mul    w0, w1, w0
    8a4:    str    w0, [sp, #20]
    8a8:    mov    w0, #0x4b0
    8ac:    str    w0, [sp, #24]    XXXXX
    8b0:    ldr    w1, [sp, #20]
    8b4:    ldr    w0, [sp, #24]    XXXXX
    8b8:    sdiv   w0, w1, w0
    8bc:    str    w0, [sp, #28]    XXXXX
    8c0:    ldr    w0, [sp, #28]    XXXXX
    8c4:    add    sp, sp, #0x20
    8c8:    ret
```

```
0000000000400544 <sum>:
    400544:    d10043ff     sub    sp, sp, #0x10
    400548:    b9000fe0     str    w0, [sp, #12]
    40054c:    b9000be1     str    w1, [sp, #8] 
    400550:    b9400fe8     ldr    w8, [sp, #12]
    400554:    b9400be9     ldr    w9, [sp, #8] 
    400558:    0b090108     add    w8, w8, w9   
    40055c:    b90007e8     str    w8, [sp, #4] 
    400560:    b94007e0     ldr    w0, [sp, #4] 
    400564:    910043ff     add    sp, sp, #0x10
    400568:    d65f03c0     ret                 
```

```
000000000000088c <interest_due>:
    88c:    sub    sp, sp, #0x20
    890:    str    w0, [sp, #12]
    894:    str    w1, [sp, #8]
    898:    ldr    w0, [sp, #12]
    89c:    ldr    w1, [sp, #8] 
    8a0:    mul    w0, w1, w0
    8a4:    str    w0, [sp, #20]
    8a8:    mov    w0, #0x4b0
    8ac:    str    w0, [sp, #24]
    8b0:    ldr    w1, [sp, #20]
    8b4:    ldr    w0, [sp, #24] 
    8b8:    sdiv   w0, w1, w0
    8bc:    str    w0, [sp, #28] 
    8c0:    ldr    w0, [sp, #28] 
    8c4:    add    sp, sp, #0x20
    8c8:    ret
```

```
0000000000400544 <sum>:
    400544:    d10043ff     sub    sp, sp, #0x10
    400548:    b9000fe0     str    w0, [sp, #12]
    40054c:    b9000be1     str    w1, [sp, #8] 
    400550:    b9400fe8     ldr    w8, [sp, #12]
    400554:    b9400be9     ldr    w9, [sp, #8] 
    400558:    0b090108     add    w8, w8, w9   
    40055c:    b90007e8     str    w8, [sp, #4] 
    400560:    b94007e0     ldr    w0, [sp, #4] 
    400564:    910043ff     add    sp, sp, #0x10
    400568:    d65f03c0     ret                 
```

* Update register assignments to eliminate loads and stores
    * Must preserve calling conventions
        * Parameters are stored in w/x0, w/x1, ...
        * Return value is stored in w/x0
        * Caller must store register values into caller's stack frame before `bl` to callee — actually only needed if values in registers are needed by caller after `bl` and callee overwrites the values in those registers
    * Example

```
000000000000088c <interest_due>:
    88c:    sub    sp, sp, #0x20    XXXXX
    8a0:    mul    w0, w1, w0       
    8a4:    str    w0, [sp, #20]    XXXXX
    8a8:    mov    w0, #0x4b0       // mov w1 #0x4b0
    8b0:    ldr    w1, [sp, #20]    XXXXX
    8b8:    sdiv   w0, w1, w0       // sdiv w0, w0, w1
    8c4:    add    sp, sp, #0x20    XXXXX
    8c8:    ret
```

```
0000000000400544 <sum>:
    400544:    d10043ff     sub    sp, sp, #0x10    XXXXX
    400548:    b9000fe0     str    w0, [sp, #12]    XXXXX
    40054c:    b9000be1     str    w1, [sp, #8]     XXXXX
    400550:    b9400fe8     ldr    w8, [sp, #12]    XXXXX
    400554:    b9400be9     ldr    w9, [sp, #8]     XXXXX
    400558:    0b090108     add    w8, w8, w9       // add w0, w0, w1
    40055c:    b90007e8     str    w8, [sp, #4]     XXXXX
    400560:    b94007e0     ldr    w0, [sp, #4]     XXXXX
    400564:    910043ff     add    sp, sp, #0x10    XXXXX
    400568:    d65f03c0     ret                 
```

## Temporal vs. spatial locality

* _What is temporal locality?_
    * Access the same data repeatedly
    * E.g., for loop variable
* _What is spatial locality?_
    * Access data with a similar scope
    * E.g., next item in array
    * E.g., local variables/parameters, which are stored in the same stack frame
* Analogies for temporal and spatial locality
    * Book storage (Dive Into Systems Section 11.3.2)
        * Temporal locality — store most frequently used books at your desk, less frequently used books on your bookshelf, and least frequently used books at the library
        * Spatial locality — checkout books on the same/nearby subjects when you go to the library
    * Groceries (pre-class questions 3 & 4)
        * Temporal locality — you store food you eat frequently in the front of the refrigerator, while you store food you eat infrequently in the back of the refrigerator
        * Spatial locality — you organize the items on your grocery list based on the aisle in which they are located
    * _With a partner, develop your own analogy for temporal and spatial locality_